# TODO
- Comments
- Show dates instead of numbers on x-axis

In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import plotly.express as px

In [96]:
data = pd.read_csv("incoming_packages.csv")

In [97]:
# Convert string dates into datetime objects
data_final = data.copy()
data_final['date'] = pd.to_datetime(data_final['date'])

In [98]:
def get_prophet_data( col:str, startdate:datetime, enddate:datetime ) -> pd.DataFrame:
    """ Get data that can be used by prophet from the specified date range. """
    prophet_data = pd.DataFrame()
    prophet_data["ds"] = data_final["date"]
    prophet_data["y"] = data_final[col]
    prophet_data = prophet_data[ (prophet_data['ds'] >= startdate) & (prophet_data['ds'] <= enddate)]

    return prophet_data

In [99]:
data.head(len(data))

,date,total_packages,cooled_packages,fire_packages,normal_packages
0,2020-01-01,1745,370,488,887
1,2020-01-02,1847,308,402,1137
2,2020-01-03,1842,314,394,1134
3,2020-01-04,664,131,181,352
4,2020-01-05,871,137,190,544
...,...,...,...,...,...
846,2022-04-26,2268,467,503,1298
847,2022-04-27,2466,432,641,1393
848,2022-04-28,2436,545,556,1335
849,2022-04-29,2302,368,547,1387


In [101]:
fig = px.line(data, x='date', y="total_packages")
fig.show()

In [ ]:
total_packages_train = get_prophet_data("total_packages", datetime(2020, 1, 1), datetime(2022, 4, 1))
total_packages_train.head(len(total_packages_train))

,ds,y
0,2020-01-01,1745
1,2020-01-02,1847
2,2020-01-03,1842
3,2020-01-04,664
4,2020-01-05,871
...,...,...
817,2022-03-28,2431
818,2022-03-29,2223
819,2022-03-30,2260
820,2022-03-31,2374


In [ ]:
# Train prophet
prophet_ = Prophet()
prophet_.fit(total_packages_train)

14:05:39 - cmdstanpy - INFO - Chain [1] start processing
14:05:39 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
# Test prophet
total_packages_test = get_prophet_data("total_packages", datetime(2022, 4, 1), datetime(2022, 4, 30))